# Amazon Review Feature Engineering


In this notebook, I will outline how features are generated. These feature files will be fed into our models later for evaluation

To generate the features I wrote a python program that would read in a configuration file and generate all feature files needed for future notebooks at the same time.

There are ~50k samples in each file

Here are the variations of features that we generated:

| Name | Description |
|------|-------------|
| BoW | Bag of works. Variations using unigram and bigram |
| TFIDF | Term Frequency - Inverse Document Frequency. Variations using unigram and bigram. Max features is set to 10k so we only retain the most frequent 10k words |
| Word2Vec | Word 2 Vec embedding. This is an average embedding of all words in the review. <br>There are 2 variations of this. If it's pretrained, it uses word2vec-google-news-300 else embeddings are trained with our corpus of reviews. Max features is set to 10k so we only retain the most frequent 10k words. <br>Also, there are a couple different version of this using unigram (ngram11) and bigram (ngram22). Max features for this is set to 300 |
| Fasttext | Fasttext embedding. This is an average embedding of all words in the review. Embedding is trained with our corpus of review words. <br>Also, there are a couple different version of this using unigram (ngram11) and bigram (ngram22). Max features for this is set to 300 |


In [1]:
# import libraries
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import logging
import util.nlp_util as nlpu

# configure logger so we can see output from the classes
logging.basicConfig(level=logging.INFO)

%matplotlib inline

BASE='20191101-feature_generator-50k'
CONFIG_DIR="../config"
CONFIG_FILE=f'{CONFIG_DIR}/{BASE}.csv'
REPORT_DIR="../reports"
REPORT_FILE=f'{REPORT_DIR}/{BASE}-report.csv'


# Running our Program To Generate File

run the following command in tools directory:
    
```
python feature_generator.py ../config/20191101-feature_generator-50k.csv
```


The program will loop through each of the entry in the configuration file and generate a new feature file.

The program dynamically uses *fn_name* column to determine which of the functions below to call.

# Our configuration file

In [2]:
config = pd.read_csv(CONFIG_FILE)
config


,data_dir,data_file,description,fn_name,lda_topics,min_df,max_df,min_ngram_range,max_ngram_range,max_features,feature_size,window_context,min_word_count,sample,iterations,feature_columns,y,status,status_date,message
0,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,bow-df_default-ngram11,generate_bow_file,NaN,NaN,NaN,1.0,1.0,10000.0,NaN,NaN,NaN,NaN,NaN,review_body,"star_rating, helpful_votes, total_votes",success,NaN,NaN
1,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,tfidf-df_default-ngram11,generate_tfidf_file,NaN,NaN,NaN,1.0,1.0,10000.0,NaN,NaN,NaN,NaN,NaN,review_body,"star_rating, helpful_votes, total_votes",success,NaN,NaN
2,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,bow-df_default-ngram22,generate_bow_file,NaN,NaN,NaN,2.0,2.0,10000.0,NaN,NaN,NaN,NaN,NaN,review_body,"star_rating, helpful_votes, total_votes",success,NaN,NaN
3,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,tfidf-df_default-ngram22,generate_tfidf_file,NaN,NaN,NaN,2.0,2.0,10000.0,NaN,NaN,NaN,NaN,NaN,review_body,"star_rating, helpful_votes, total_votes",success,NaN,NaN
4,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,word2vec_pretrained-df_none-ngram_none,generate_word2vec_file,NaN,NaN,NaN,NaN,NaN,NaN,100.0,5.0,5.0,0.001,5.0,review_body,"star_rating, helpful_votes, total_votes",success,NaN,NaN
5,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,word2vec-df_none-ngram_none,generate_word2vec_file,NaN,NaN,NaN,NaN,NaN,NaN,100.0,5.0,5.0,0.001,5.0,review_body,"star_rating, helpful_votes, total_votes",success,2019-11-02 09:46:59,NaN
6,../dataset/amazon_reviews,amazon_reviews_us_Wireless_v1_00-50k-preproces...,fasttext-df_none-ngram_none,generate_fasttext_file,NaN,NaN,NaN,NaN,NaN,NaN,100.0,5.0,5.0,0.001,5.0,review_body,"star_rating, helpful_votes, total_votes",success,2019-11-02 09:54:01,NaN


# BoW

Here is the code for generating BoW features

In [3]:
??nlpu.generate_bow_file

Signature:
nlpu.generate_bow_file(
    x: pandas.core.frame.DataFrame,
    y: pandas.core.series.Series,
    feature_column: str,
    description: str,
    lda_topics: int = None,
    min_df: float = 1,
    max_df: float = 1.0,
    min_ngram_range=1,
    max_ngram_range=1,
    max_features=None,
) -> (<class 'pandas.core.frame.DataFrame'>, <class 'float'>, <class 'float'>)
Source:   
def generate_bow_file(x: pd.DataFrame,
                      y: pd.Series,
                      feature_column: str,
                      description: str,
                      lda_topics: int = None,
                      min_df: float = 1,
                      max_df: float = 1.,
                      min_ngram_range=1,
                      max_ngram_range=1,
                      max_features=None
                      ) -> (pd.DataFrame, float, float):
    """
    encode feature_column using BoW

    :param x: Df with data to process
    :param y: labels
    :param feature_column: column to conver

# TF-IDF

In [4]:
??nlpu.generate_tfidf_file

Signature:
nlpu.generate_tfidf_file(
    x: pandas.core.frame.DataFrame,
    y: pandas.core.series.Series,
    feature_column: str,
    description: str,
    min_df: float = 1,
    max_df: float = 1.0,
    min_ngram_range=1,
    max_ngram_range=1,
    lda_topics: int = None,
    max_features=None,
) -> pandas.core.frame.DataFrame
Source:   
def generate_tfidf_file(x: pd.DataFrame,
                        y: pd.Series,
                        feature_column: str,
                        description: str,
                        min_df: float = 1,
                        max_df: float = 1.,
                        min_ngram_range=1,
                        max_ngram_range=1,
                        lda_topics: int = None,
                        max_features=None
                        ) -> pd.DataFrame:
    """
    generate TFIDF encoding from feature_column

    :param x: Df with data to process
    :param y: labels
    :param feature_column: column to convert to BoW
    :param descri

# Feature Engineering Based on Embeddings

Embeddings - each word is mapped to a vector space that represents the relationship of different words. The distance of the vectors represents how close each word is with each other.


### Word2Vec

In [5]:
??nlpu.generate_word2vec_file

Signature:
nlpu.generate_word2vec_file(
    x: pandas.core.frame.DataFrame,
    y: pandas.core.series.Series,
    description: str,
    feature_column: str,
    timer: util.time_util.Timer = None,
    feature_size: int = 100,
    window_context: int = 5,
    min_word_count: int = 5,
    sample: float = 0.001,
    iterations: int = 5,
)
Source:   
def generate_word2vec_file(x: pd.DataFrame,
                           y: pd.Series,
                           description: str,
                           feature_column: str,
                           timer: Timer = None,
                           feature_size: int = 100,
                           window_context: int = 5,
                           min_word_count: int = 5,
                           sample: float = 0.001,
                           iterations: int = 5,
                           ):
    """
    generate features using word2vec
    :param x:
    :param y:
    :param description:
    :param feature_size:
    :param window_c

### Fasttext

In [6]:
??nlpu.generate_fasttext_file

Signature:
nlpu.generate_fasttext_file(
    x: pandas.core.frame.DataFrame,
    y: pandas.core.series.Series,
    description: str,
    feature_column: str,
    timer: util.time_util.Timer = None,
    feature_size: int = 100,
    window_context: int = 5,
    min_word_count: int = 5,
    sample: float = 0.001,
    iterations: int = 5,
)
Source:   
def generate_fasttext_file(x: pd.DataFrame,
                           y: pd.Series,
                           description: str,
                           feature_column: str,
                           timer: Timer = None,
                           feature_size: int = 100,
                           window_context: int = 5,
                           min_word_count: int = 5,
                           sample: float = 0.001,
                           iterations: int = 5,
                           ):
    """
    generate features using fasttext embedding

    https://radimrehurek.com/gensim/models/fasttext.html

    :param x:
    :param y:

# Average Embedding for Word2Vec and Fasttext

For word2vec and fasttext. I took an average embedding approach.

For every word that is in the corpus, I averaged out the embeddings for each word to come up with the final feature vector for the review

If a word is not found the vocabulary of the trainer (ie, pre-trained embedding), that entry is removed from our training examples

In [7]:
??nlpu.get_average_embedding

Signature: nlpu.get_average_embedding(embedding, review)
Source:   
def get_average_embedding(embedding, review):
    """
    returns a list of word vectors for all words in review
    then average them to return a final vector

    :param embedding: embedding object - will be either Fasttext or Word2Vec
    :param review: review text
    :return:
    """
    log.debug(f'Getting average embedding for: [{review}]')

    wpt = WordPunctTokenizer()
    # word_vectors = [embedding.wv.get_vector(word) for word in wpt.tokenize(review)]
    word_vectors = [embedding.wv.get_vector(word) for word in wpt.tokenize(review) if word in embedding.wv.vocab]
    log.debug(f'word_vector shape [{np.shape(word_vectors)}]')

    # return average all word vectors to come up with final vector for the review
    # since we are using pre-trained embedding, we may not be able to find all the words
    if np.shape(word_vectors)[0] > 1:
        return np.average(word_vectors, axis=0)
    return None
File:      ~/

# The following files were generated

In [8]:
report = pd.read_csv(REPORT_FILE)
sorted(report.outfile.tolist())

['../dataset/feature_files/review_body-bow-df_default-ngram11-49784-10000-nolda.csv',
 '../dataset/feature_files/review_body-bow-df_default-ngram22-49784-10000-nolda.csv',
 '../dataset/feature_files/review_body-fasttext-df_none-ngram_none-47523-100-nolda.csv',
 '../dataset/feature_files/review_body-tfidf-df_default-ngram11-49784-10000-nolda.csv',
 '../dataset/feature_files/review_body-tfidf-df_default-ngram22-49784-10000-nolda.csv',
 '../dataset/feature_files/review_body-word2vec-df_none-ngram_none-47523-100-nolda.csv',
 '../dataset/feature_files/review_body-word2vec_pretrained-df_none-ngram_none-47542-300-nolda.csv']

### Reading the file name

{feature column}-{feature engineering technique}-{df parameter}-{ngram parameter}-{# of output samples}-{# of features}-{whether we appended LDA features}.csv

So for

```
review_body-bow-df_default-ngram11-49784-10000-nolda.csv
```

| Info | Description |
|------|-------------|
| feature column | review_body |
| feature engineering technique | BoW |
| df parameters | default - no specified |
| ngram parameters | 11 = (1,1) or unigram <br> 22 = (2,2) or bigram |
| output samples | 49784 |
| features | 10000 |